In [7]:
import pandas as pd
import pickle
import numpy as np

# Carregar o modelo salvo
with open('modelo_preditivo_imdb.pkl', 'rb') as file:
    model_pipeline = pickle.load(file)

# Carregar o dataset original para obter os dados do filme
df_original = pd.read_csv('desafio_indicium_imdb.csv')

# Criar coluna Stars_all (igual ao que foi feito no treino)
star_cols = [col for col in df_original.columns if col.lower().startswith('star')]
if star_cols:
    df_original['Stars_all'] = df_original[star_cols].fillna('').agg(' '.join, axis=1)
else:
    df_original['Stars_all'] = ''


def prever_nota_filme(titulo_filme):
    """
    Prevê a nota IMDB para um filme com base em seu título.
    """
    filme_teste = df_original[df_original['Series_Title'] == titulo_filme].copy()

    if filme_teste.empty:
        print(f"Erro: O filme '{titulo_filme}' não foi encontrado no dataset.")
        return

    # Tratar Gross
    gross_value = float(filme_teste['Gross'].iloc[0].replace(',', '')) if pd.notna(filme_teste['Gross'].iloc[0]) else 0.0
    genre_main_value = filme_teste['Genre'].iloc[0].split(',')[0].strip()

    # Preparar as features exatamente como no treino
    filme_teste_features_clean = pd.DataFrame({
        'Genre_main': [genre_main_value],
        'Director': [filme_teste['Director'].iloc[0]],
        'Gross_log': [np.log1p(gross_value)],
        'No_of_Votes': [filme_teste['No_of_Votes'].iloc[0]],
        'Meta_score': [filme_teste['Meta_score'].iloc[0]],
        'Overview': [filme_teste['Overview'].iloc[0]],
        'Stars_all': [filme_teste['Stars_all'].iloc[0]]
    })

    # Fazer a previsão
    previsao_nota = model_pipeline.predict(filme_teste_features_clean)

    # Imprimir o resultado
    print(f"\n--- Análise para o filme: '{titulo_filme}' ---")
    print(f"Nota IMDB real de {titulo_filme}: {filme_teste['IMDB_Rating'].iloc[0]}")
    print(f"Nota IMDB prevista pelo modelo para {titulo_filme}: {previsao_nota[0]:.2f}")


# Exemplo de uso da função
prever_nota_filme('The Godfather')

prever_nota_filme('Pulp Fiction')

prever_nota_filme('Filme Inexistente')


--- Análise para o filme: 'The Godfather' ---
Nota IMDB real de The Godfather: 9.2
Nota IMDB prevista pelo modelo para The Godfather: 9.06

--- Análise para o filme: 'Pulp Fiction' ---
Nota IMDB real de Pulp Fiction: 8.9
Nota IMDB prevista pelo modelo para Pulp Fiction: 8.89
Erro: O filme 'Filme Inexistente' não foi encontrado no dataset.
